<a href="https://colab.research.google.com/github/AnthonyJFeola/DS-Unit-4-Sprint-1-NLP/blob/main/module3-document-classification/DSPT6_Kaggle_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM, SpatialDropout1D

max_features = 20000
maxlen = 1000
batch_size = 80

In [ ]:
train = pd.read_csv('train.csv')
print(train.shape)
train.head()

(4087, 3)


,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [ ]:
test = pd.read_csv('test.csv')
print(test.shape)
test.head()

(1022, 2)


,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['description'])

In [ ]:
x_train = tokenizer.texts_to_sequences(train['description'])
y_train = np.array(train['ratingCategory'])

In [ ]:
x_test = tokenizer.texts_to_sequences(test['description'])

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
model = Sequential()

model.add(Embedding(max_features, 128))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, None, 128)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 12)                780       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 39        
Total params: 2,700,659
Trainable params: 2,700,659
Non-trainable params: 0
____________________________________________

In [ ]:
output = model.fit(x_train, y_train,
          batch_size=batch_size, 
          epochs=5)

Epoch 1/5
52/52 [==============================] - 288s 6s/step - loss: 0.7504 - accuracy: 0.6995
Epoch 2/5
52/52 [==============================] - 288s 6s/step - loss: 0.5814 - accuracy: 0.7235
Epoch 3/5
52/52 [==============================] - 282s 5s/step - loss: 0.4330 - accuracy: 0.8189
Epoch 4/5
52/52 [==============================] - 280s 5s/step - loss: 0.2855 - accuracy: 0.8943
Epoch 5/5
52/52 [==============================] - 281s 5s/step - loss: 0.1865 - accuracy: 0.9356


In [ ]:
pred = model.predict(x_test)

In [ ]:
pred_labels = []

for array in pred:
  pred_labels.append(np.argmax(array))

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred_labels})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,0
4,2306,1


In [ ]:
subNumber = 0

In [ ]:
submission.to_csv(f'Kaggle_Submission{subNumber}.csv', index=False)